In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Tornar os números mais legiveis
pd.options.display.float_format = '{:20.2f}'.format

# Mostrar todas as colunas
pd.set_option('display.max_columns', 999)

### Análise Exploratória

In [3]:
df = pd.read_excel("data/online_retail_II.xlsx", sheet_name=0)

In [6]:
df.head(5)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [8]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,525461.00,525461,525461.00,417534.00
mean,10.34,2010-06-28 11:37:36.845017856,4.69,15360.65
min,-9600.00,2009-12-01 07:45:00,-53594.36,12346.00
25%,1.00,2010-03-21 12:20:00,1.25,13983.00
50%,3.00,2010-07-06 09:51:00,2.10,15311.00
75%,10.00,2010-10-15 12:45:00,4.21,16799.00
max,19152.00,2010-12-09 20:01:00,25111.09,18287.00
std,107.42,NaN,146.13,1680.81


In [9]:
df.describe(include='O')

,Invoice,StockCode,Description,Country
count,525461,525461,522533,525461
unique,28816,4632,4681,40
top,537434,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
freq,675,3516,3549,485852


##### Customer ID

In [10]:
df[df['Customer ID']. isna()].head(5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.00,NaN,United Kingdom
283,489463,71477,short,-240,2009-12-01 10:52:00,0.00,NaN,United Kingdom
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.00,NaN,United Kingdom
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.00,NaN,United Kingdom
577,489525,85226C,BLUE PULL BACK RACING CAR,1,2009-12-01 11:49:00,0.55,NaN,United Kingdom


Como estamos fazendo um estudo sobre o hábito dos consumidores, entradas que não possuem um 'Customer ID' serão eliminadas do banco de dados


##### Quantity

In [11]:
df[df["Quantity"] < 0].head(5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.00,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.00,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.00,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.00,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.00,Australia


Como não faz sentido um item ser vendido uma quantidade negativa de vezes, essas entradas também serão removidas do banco de dados

##### Invoice

Sobre a coluna "Invoice", ela corresponde ao número da fatura, um valor único para cada pedido. É composta de 6 digitos numéricos, podendo conter a letra 'C' no início, indicando um cancelamento. Vamos verificar se todas as observações estão de acordo com essa regra.

In [19]:
df["Invoice"] = df["Invoice"].astype("str")
df[df["Invoice"].str.match("^\\d{6}$") == False].head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.00,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.00,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.00,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.00,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.00,Australia


In [20]:
df["Invoice"] = df["Invoice"].astype("str")

df[df["Invoice"].str.match("^\\d{6}$") == False].head()
# usando expressões regex, estamos verificando se a string começa (^) com 6 digitos 
# (\\d) e termina com apenas esses digitos e mais nenhum caractere.

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.00,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.00,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.00,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.00,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.00,Australia


Percebemos que existem valores com a letra "C" no início do número da fatura, mas para verificar se existem outros casos, podemos remover os digitos númericos [0-9] e verificar quantos valores únicos existem, se todos os dados estiverem seguindo as regras, devemos ter apenas os valores ["", "C"].

In [16]:
df["Invoice"].str.replace("[0-9]", "", regex=True).unique()
# novamente usando regex, acessamos o atributo string do objeto DataFrame, 
# e substituimos, na string, os valores de 0 a 9 pela string vazia (""), e contamos quantos
# valores são únicos

array(['', 'C', 'A'], dtype=object)

In [18]:
df[df["Invoice"].str.contains("A")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
179403,A506401,B,Adjust bad debt,1,2010-04-29 13:36:00,-53594.36,NaN,United Kingdom
276274,A516228,B,Adjust bad debt,1,2010-07-19 11:24:00,-44031.79,NaN,United Kingdom
403472,A528059,B,Adjust bad debt,1,2010-10-20 12:04:00,-38925.87,NaN,United Kingdom


Percebemos que existem valores de "Invoice" que contém o caractere "A", observando esses valores, percebemos que todos possuem o "CustomerID" = NaN, logo serão rertirados do banco de dados.

##### StockCode

A variável "StockCode" é um numério inteiro único de 5 digitos para cada produto distinto

In [32]:
df["StockCode"] = df["StockCode"].astype('str')

df[df['StockCode'].str.match("^\\d{5}$") == False].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
12,489436,48173C,DOOR MAT BLACK FLOCK,10,2009-12-01 09:06:00,5.95,13078.00,United Kingdom
23,489436,35004B,SET OF 3 BLACK FLYING DUCKS,12,2009-12-01 09:06:00,4.65,13078.00,United Kingdom
28,489436,84596F,SMALL MARSHMALLOWS PINK BOWL,8,2009-12-01 09:06:00,1.25,13078.00,United Kingdom


Existem outros 'StockCode's que possuem mais de cinco digitos numéricos, verificando todos que existem, temos:

In [33]:
df[(df['StockCode'].str.match("^\\d{5}$") == False) &
   (df['StockCode'].str.match("^\\d{5}[a-zA-Z]+$") == False)]['StockCode'].unique()
# estamos verificando em que situações os valores da variável StockCode não possui
# cinco digitos e não possui letras[a-zA-z] no final da string, o '+' serve para indicar que
# podem existir letras que se repetem

array(['POST', 'D', 'DCGS0058', 'DCGS0068', 'DOT', 'M', 'DCGS0004',
       'DCGS0076', 'C2', 'BANK CHARGES', 'DCGS0003', 'TEST001',
       'gift_0001_80', 'DCGS0072', 'gift_0001_20', 'DCGS0044', 'TEST002',
       'gift_0001_10', 'gift_0001_50', 'DCGS0066N', 'gift_0001_30',
       'PADS', 'ADJUST', 'gift_0001_40', 'gift_0001_60', 'gift_0001_70',
       'gift_0001_90', 'DCGSSGIRL', 'DCGS0006', 'DCGS0016', 'DCGS0027',
       'DCGS0036', 'DCGS0039', 'DCGS0060', 'DCGS0056', 'DCGS0059', 'GIFT',
       'DCGSLBOY', 'm', 'DCGS0053', 'DCGS0062', 'DCGS0037', 'DCGSSBOY',
       'DCGSLGIRL', 'S', 'DCGS0069', 'DCGS0070', 'DCGS0075', 'B',
       'DCGS0041', 'ADJUST2', '47503J ', 'C3', 'SP1002', 'AMAZONFEE'],
      dtype=object)

É necessário verificar todos esses casos.

In [34]:
df[df["StockCode"].str.contains("^DOT")].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
2379,489597,DOT,DOTCOM POSTAGE,1,2009-12-01 14:28:00,647.19,NaN,United Kingdom
2539,489600,DOT,DOTCOM POSTAGE,1,2009-12-01 14:43:00,55.96,NaN,United Kingdom
2551,489601,DOT,DOTCOM POSTAGE,1,2009-12-01 14:44:00,68.39,NaN,United Kingdom
2571,489602,DOT,DOTCOM POSTAGE,1,2009-12-01 14:45:00,59.35,NaN,United Kingdom
2619,489603,DOT,DOTCOM POSTAGE,1,2009-12-01 14:46:00,42.39,NaN,United Kingdom


Nota:

StockCode deve seguir o padrão [0-9]{5} mas aparenta apresentar valores legítimos para o padrão [0-9]{5}[a-zA-Z]+
Além disso contém outros valores da forma:

| Código | Descrição | Ação | 
|---------|-------------|----------|
| DCGS | CustomerId nulo | Excluir| 
| D | Representa desconto de valores | Excluir | 
| DOT | Custo de postagem | Exclurir | 
| M ou m | Transações manuais | Excluir | 
| C2 | Custo de transporte (?) | Excluir | 
| C3 | Não sei, apenas uma transação | Excluir | 
| BANK CHARGES ou B | Tarifas de Banco | Excluir| 
| S | Amostras enviadas aos clientes | Excluir | 
| TESTXXX | Dados de teste, não é válido | Excluir | 
| gift__XXX | Compras com "gift cards" | Excluir | 
| PADS | Código de estoque para preenchimento | Incluir | 
| SP1002 | Pedidos especiais para itens, apenas 3 transações | Excluir| 
| AMAZONFEE | Taxa para transporte com a Amazon | Excluir |
| ADJUSTX | Ajuste manual feito pelos Adms | Excluir |

### Limpando o banco de dados

In [ ]:
cleaned_df = df.copy()

##### Invoice

In [ ]:
cleaned_df["Invoice"] = cleaned_df["Invoice"].astype('str')
# transformando todos os valores de "Invoice" em strings


# mask é usado como uma expressão para filtrar os dados no pandas!
mask = (
    cleaned_df["Invoice"].str.match("^\\d{6}$") == True
)

cleaned_df = cleaned_df[mask]
# queremos que cleaned_df seja um DataFrame que respeita as regras impostas pelo
# filtro "mask"

##### StockCode

In [ ]:
cleaned_df["StockCode"] = cleaned_df["StockCode"].astype("str")

# queremos os valores de "StockCode" que contenham 5 digitos, 5 digitos seguidos por
# letras, e que contenham o código "PADS". "|" serve com "ou"

mask = (
    (cleaned_df["StockCode"].str.match("^\\d{5}$") == True)
    | (cleaned_df["StockCode"].str.match("^\\d{5}[a-zA-Z]+$"))
    | (cleaned_df["StockCode"].str.match("^PADS$"))
)

cleaned_df = cleaned_df[mask]

##### Customerr ID

In [39]:
cleaned_df.dropna(subset=["Customer ID"], inplace=True)

In [40]:
cleaned_df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,407695.00,407695,407695.00,407695.00
mean,13.59,2010-07-01 10:10:10.782177792,3.29,15368.50
min,1.00,2009-12-01 07:45:00,0.00,12346.00
25%,2.00,2010-03-26 14:01:00,1.25,13997.00
50%,5.00,2010-07-09 15:46:00,1.95,15321.00
75%,12.00,2010-10-14 17:09:00,3.75,16812.00
max,19152.00,2010-12-09 20:01:00,10953.50,18287.00
std,96.84,NaN,34.76,1679.80


##### Price

Aparentam existir valores de preço iguais a zero nas transações.

In [48]:
cleaned_df = cleaned_df[cleaned_df["Price"] > 0]

In [49]:
cleaned_df["Price"].min()

0.001

In [50]:
len(cleaned_df[cleaned_df["Price"] == 0])

0

In [51]:
cleaned_df["Price"].min()

0.001

Notamos que não existem preços de transações iguais a zero, mesmo que possuam valores muito pequenos, todas são maiores que zero, então serão mantidas no banco de dados.

In [53]:
print(len(cleaned_df) / len(df))

0.7758216118798541


Após a limpeza dos dados, perdemos aproximadamente 23% dos dados.